

## Assignment: Fine-tuning DistilBERT for Sentiment Analysis

### 🔍 What is DistilBERT?  
DistilBERT is a **lightweight, faster, and smaller** version of BERT, designed to maintain **97% of BERT’s performance** while being **60% faster** and using **40% fewer parameters**. It achieves this through a process called **knowledge distillation**, where it learns from a larger BERT model while retaining essential knowledge.  

🚀 **Why use DistilBERT?**  
- **Faster Inference** – Great for real-time applications!  
- **Smaller Model Size** – Ideal for deployment on edge devices.  
- **Competitive Accuracy** – Performs nearly as well as BERT on many NLP tasks.  

---

__Objective__:
Fine-tune the DistilBERT model on the IMDb movie reviews dataset to classify reviews as positive or negative.

In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pandas as pd
import torch




In [ ]:
# Load IMDb dataset from Hugging Face
dataset = load_dataset("imdb")

In [ ]:
# Load the pretrained DistilBERT and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the dataset

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

0.8184822389946661


In [ ]:
## Split dataset into train and test

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

0.4445498195712254


In [ ]:
# Evaluate pre-trained DistilBERT model on IMDb test-set
# Note: Since classifier head is randomly initialized, performance is expected to be closed to random

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

In [ ]:
# Define training arguments

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

In [ ]:
# Fine-tune DistilBERT model

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

In [ ]:
# Re-evaluate the fine-tuned model

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT

In [ ]:
# Report the results with pre-trained model and finetuned model

# Fill code here

#The below random line needs to be there at the end of the cell
import random; print(random.random()) # DO NOT REMOVE THIS LINE THAT HAS RANDOM IN IT